<a href="https://colab.research.google.com/github/ykitaguchi77/Strabismus_AI_project/blob/main/DataSplit(stratified_one_subject_leave_out).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import codecs
import pandas as pd

gla_ortho_path = r"F:\先天性緑内障\data_list\Disease_ortho.csv"
gla_eso_path = r"F:\先天性緑内障\data_list\Disease_ET.csv"
gla_exo_path = r"F:\先天性緑内障\data_list\Disease_XT.csv"
cont_ortho_path = r"F:\先天性緑内障\data_list\Control_ortho.csv"
cont_eso_path = r"F:\先天性緑内障\data_list\Control_ET.csv"
cont_exo_path = r"F:\先天性緑内障\data_list\Control_XT.csv"

def opencsv(path, classname):
    #with codecs.open(path, "r", "utf-8", "ignore") as file:
    with codecs.open(path, "r", "Shift-JIS", "ignore") as file:
        df = pd.read_csv(file, index_col=None, header=None)
        df.insert(0, 'ID', '')
        df.insert(0, 'classes', '')
        for row in range(len(df)):
            df.iloc[row,0] = classname #class
            df.iloc[row,1] = os.path.basename(df.iloc[row,2]).split("_")[0] #ID
    return df

df_gla_ortho = opencsv(gla_ortho_path, "gla_ortho") 
df_gla_eso = opencsv(gla_eso_path, "gla_eso")
df_gla_exo = opencsv(gla_exo_path, "gla_exo")
df_cont_ortho = opencsv(cont_ortho_path, "cont_ortho")
df_cont_eso = opencsv(cont_eso_path, "gla_eso")
df_cont_exo = opencsv(cont_exo_path, "gla_exo")

df_cont_eso

,classes,ID,0
0,gla_eso,1047,F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりな...
1,gla_eso,1082,F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりな...
2,gla_eso,1097,F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりな...
3,gla_eso,1107,F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりな...
4,gla_eso,1120,F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりな...
5,gla_eso,1197,F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりな...
6,gla_eso,1212,F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりな...
7,gla_eso,1232,F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりな...
8,gla_eso,1290,F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりな...
9,gla_eso,1306,F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりな...


In [25]:
import numpy as np
from sklearn.model_selection import KFold

X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(X):
    print("%s %s" % (train, test))

[2 3] [0 1]
[0 1] [2 3]
